In [ ]:
# server.py
import pyarrow as pa
import pyarrow.flight
import pyarrow.parquet
import pathlib

class FlightServer(pa.flight.FlightServerBase):

    def __init__(self, location="grpc://0.0.0.0:8816", repo="./datasets"):
        super().__init__(location)
        self._location = location
        self._repo = pathlib.Path(repo)

    def _make_flight_info(self, dataset):
        dataset_path = self._repo / dataset
        metadata = pa.parquet.read_metadata(dataset_path)
        descriptor = pa.flight.FlightDescriptor.for_path(dataset.encode())
        endpoints = [pa.flight.FlightEndpoint(dataset, [self._location])]
        return pa.flight.FlightInfo(
            pa.parquet.read_schema(dataset_path),
            descriptor,
            endpoints,
            metadata.num_rows,
            metadata.serialized_size
        )

    def list_flights(self, context, criteria):
        for dataset in self._repo.iterdir():
            yield self._make_flight_info(dataset.name)

    def get_flight_info(self, context, descriptor):
        return self._make_flight_info(descriptor.path[0].decode())

    def do_put(self, context, descriptor, reader, writer):
        dataset = descriptor.path[0].decode()
        dataset_path = self._repo / dataset
        data_table = reader.read_all()
        pa.parquet.write_table(data_table, dataset_path)

    def do_get(self, context, ticket):
        dataset = ticket.ticket.decode()
        dataset_path = self._repo / dataset
        return pa.flight.RecordBatchStream(pa.parquet.read_table(dataset_path))

    def list_actions(self, context):
        return [("drop_dataset", "Delete a dataset.")]

    def do_action(self, context, action):
        if action.type == "drop_dataset":
            self._drop_dataset(action.body.to_pybytes().decode())
        else:
            raise NotImplementedError

    def _drop_dataset(self, dataset):
        (self._repo / dataset).unlink()

if __name__ == "__main__":
    server = FlightServer()
    server._repo.mkdir(exist_ok=True)
    server.serve()
